##### Copyright 2018 Les auteurs TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Mettre à niveau automatiquement le code vers TensorFlow 2

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/upgrade"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Voir sur TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/upgrade.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Exécuter dans Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/upgrade.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Voir la source sur GitHub</a></td>
  <td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/upgrade.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Télécharger le cahier</a></td>
</table>


TensorFlow 2.0 inclut de nombreuses modifications d'API, telles que la réorganisation des arguments, le changement de nom des symboles et la modification des valeurs par défaut des paramètres. Effectuer manuellement toutes ces modifications serait fastidieux et sujet à erreur. Pour rationaliser les modifications et rendre votre transition vers TF 2.0 aussi transparente que possible, l'équipe TensorFlow a créé l'utilitaire `tf_upgrade_v2` pour faciliter la transition du code hérité vers la nouvelle API.

Remarque: `tf_upgrade_v2` est installé automatiquement pour TensorFlow 1.13 et versions ultérieures (y compris toutes les versions de TF 2.0).

L'utilisation typique est comme ceci:

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">tf_upgrade_v2 \<br>--intree mon_projet / \<br>--outtree my_project_v2 / \<br>--reportfile report.txt<br></pre>

Il accélérera votre processus de mise à niveau en convertissant les scripts Python TensorFlow 1.x existants en TensorFlow 2.0.

Le script de conversion automatise autant que possible, mais il existe toujours des modifications syntaxiques et stylistiques qui ne peuvent pas être effectuées par le script.

## Modules de compatibilité

Certains symboles API ne peuvent pas être mis à niveau simplement en utilisant un remplacement de chaîne. Pour vous assurer que votre code est toujours pris en charge dans TensorFlow 2.0, le script de mise à niveau comprend un module `compat.v1` . Ce module remplace les symboles TF 1.x comme `tf.foo` par la référence équivalente `tf.compat.v1.foo` . Bien que le module de compatibilité soit agréable, nous vous recommandons de relire manuellement les remplacements et de les migrer vers de nouvelles API dans l'espace de noms `tf.*` lieu de l'espace de noms `tf.compat.v1` aussi rapidement que possible.

En raison des obsolescences du module TensorFlow 2.x (par exemple, `tf.flags` et `tf.contrib` ), certaines modifications ne peuvent pas être contournées en passant à `compat.v1` . La mise à niveau de ce code peut nécessiter l'utilisation d'une bibliothèque supplémentaire (par exemple, [`absl.flags`](https://github.com/abseil/abseil-py) ) ou le passage à un package dans [tensorflow / addons](http://www.github.com/tensorflow/addons) .


## Processus de mise à niveau recommandé

Le reste de ce guide montre comment utiliser le script de mise à niveau. Bien que le script de mise à niveau soit facile à utiliser, il est fortement recommandé d'utiliser le script dans le cadre du processus suivant:

1. **Test unitaire** : assurez-vous que le code que vous mettez à niveau dispose d'une suite de tests unitaires avec une couverture raisonnable. Ceci est du code Python, donc le langage ne vous protégera pas de nombreuses classes d'erreurs. Assurez-vous également que toute dépendance dont vous disposez a déjà été mise à niveau pour être compatible avec TensorFlow 2.0.

2. **Installez TensorFlow 1.14** : mettez à niveau votre TensorFlow vers la dernière version de TensorFlow 1.x, au moins 1.14. Cela inclut l'API TensorFlow 2.0 finale dans `tf.compat.v2` .

3. **Test With 1.14**: Ensure your unit tests pass at this point. You’ll be running them repeatedly as you upgrade so starting from green is important.

4. **Run the upgrade script**: Run `tf_upgrade_v2` on your entire source tree, tests included. This will upgrade your code to a format where it only uses symbols available in TensorFlow 2.0. Deprecated symbols will be accessed with `tf.compat.v1`. These will eventually require manual attention, but not immediately.

5. **Exécutez les tests convertis avec TensorFlow 1.14** : votre code devrait toujours fonctionner correctement dans TensorFlow 1.14. Exécutez à nouveau vos tests unitaires. Toute erreur dans vos tests signifie qu'il y a un bogue dans le script de mise à niveau. [Faites-le nous savoir](https://github.com/tensorflow/tensorflow/issues) .

6. **Recherchez des avertissements et des erreurs dans le rapport de mise à niveau** : le script écrit un fichier de rapport qui explique toutes les conversions que vous devez revérifier ou toute action manuelle que vous devez effectuer. Par exemple: toutes les instances restantes de contrib nécessiteront une action manuelle pour être supprimées. Veuillez consulter [le RFC pour plus d'instructions](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md) .

7. **Installer TensorFlow 2.0** : à ce stade, il devrait être sûr de passer à TensorFlow 2.0

8. **Test with `v1.disable_v2_behavior`**: Re-running your tests with al `v1.disable_v2_behavior()` in the tests main function should give the same results as running under 1.14.

9. **Activer le comportement V2** : Maintenant que vos tests fonctionnent à l'aide de l'API v2, vous pouvez commencer à envisager d'activer le comportement v2. Selon la façon dont votre code est écrit, cela peut nécessiter certaines modifications. Voir le [guide de migration](migrate.ipynb) pour plus de détails.

## Utilisation du script de mise à niveau


### Installer

Avant de commencer, assurez-vous que TensorlFlow 2.0 est installé.

In [0]:
import tensorflow as tf

print(tf.__version__)

Clone the [tensorflow/models](https://github.com/tensorflow/models) git repository so you have some code to test on:

In [0]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

### Lisez l'aide

Le script doit être installé avec TensorFlow. Voici l'aide intégrée:

In [0]:
!tf_upgrade_v2 -h

### Exemple de code TF1

Voici un simple script TensorFlow 1.0:

In [0]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

Avec TensorFlow 2.0 installé, il ne fonctionne pas:

In [0]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

### Un seul fichier

Le script de mise à niveau peut être exécuté sur un seul fichier Python:

In [0]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

Le script affichera des erreurs s'il ne trouve pas de correctif pour le code. 

### Arborescence de répertoires

Les projets typiques, y compris cet exemple simple, utiliseront beaucoup plus d'un fichier. Vous souhaitez généralement mettre à niveau un package entier, de sorte que le script peut également être exécuté sur une arborescence de répertoires:

In [0]:
# upgrade the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

Notez le seul avertissement concernant la fonction `dataset.make_one_shot_iterator` .

Maintenant, le script fonctionne avec TensorFlow 2.0:

Notez qu'en raison du module `tf.compat.v1` , le script converti s'exécutera également dans TensorFlow 1.14. 

In [0]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## Rapport détaillé

Le script rapporte également une liste des modifications détaillées. Dans cet exemple, il a trouvé une transformation potentiellement dangereuse et a inclus un avertissement en haut du fichier: 

In [0]:
!head -n 20 tree_report.txt

Notez à nouveau le seul avertissement concernant la `Dataset.make_one_shot_iterator function` .

Dans d'autres cas, la sortie expliquera le raisonnement pour des changements non triviaux:

In [0]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [0]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [0]:
!cat dropout_report.txt

Voici le contenu du fichier modifié, notez comment le script ajoute des noms d'arguments pour gérer les arguments déplacés et renommés:

In [0]:
!cat dropout_v2.py

Un projet plus important peut contenir quelques erreurs. Par exemple, convertissez le modèle deeplab:

In [0]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

Il a produit les fichiers de sortie:

In [0]:
!ls deeplab_v2

Mais il y avait des erreurs. Le rapport vous aidera à identifier ce que vous devez corriger avant que cela ne s'exécute. Voici les trois premières erreurs:

In [0]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## Mode "sécurité"

Le script de conversion a également un mode de `SAFETY` moins invasif qui modifie simplement les importations pour utiliser le module `tensorflow.compat.v1` :

In [0]:
!cat dropout.py

In [0]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [0]:
!cat dropout_v2_safe.py

Comme vous pouvez le voir, cela ne met pas à niveau votre code, mais permet au code TensorFlow 1 de s'exécuter dans TensorFlow 2

## Avertissements

- Ne mettez pas à jour des parties de votre code manuellement avant d'exécuter ce script. En particulier, les fonctions qui ont eu des arguments réorganisés comme `tf.argmax` ou `tf.batch_to_space` entraînent l'ajout incorrect d'arguments de mots clés qui ne correspondent pas à votre code existant.

- Le script suppose que `tensorflow` est importé en utilisant `import tensorflow as tf` .

- Ce script ne réorganise pas les arguments. Au lieu de cela, le script ajoute des arguments de mots clés aux fonctions dont les arguments ont été réorganisés.

- Consultez [tf2up.ml](http://tf2up.ml) pour un outil pratique pour mettre à niveau les blocs-notes Jupyter et les fichiers Python dans un référentiel GitHub.

Pour signaler des bogues de script de mise à niveau ou faire des demandes de fonctionnalités, veuillez signaler un problème sur [GitHub](https://github.com/tensorflow/tensorflow/issues) . Et si vous testez TensorFlow 2.0, nous voulons en entendre parler! Rejoignez la [communauté TF 2.0 Testing](https://groups.google.com/a/tensorflow.org/forum/#!forum/testing) et envoyez vos questions et discussions à [testing@tensorflow.org](mailto:testing@tensorflow.org) .